## Introduction
Running the storage valuation models in Python first involves creating an instance of the CmdtyStorage type. In order to represent the diverse characteristics of real physical storage facilities there are many different ways of creating CmdtyStorage instances. This workbook gives an overview of the different options available.

In [7]:
# Common Imports
from cmdty_storage import CmdtyStorage
import pandas as pd

### The Most Simple Storage and Common Arguments
The code below creates the most simple CmdtyStorage instance, with constant injection/withdrawal rates and costs.

In [5]:
simple_storage = CmdtyStorage(
    freq='D',
    storage_start = '2021-04-01',
    storage_end = '2022-04-01',
    injection_cost = 0.01,
    withdrawal_cost = 0.025,
    min_inventory = 0.0,
    max_inventory = 1500.0,
    max_injection_rate = 25.5,
    max_withdrawal_rate = 30.9
)

The following arguments are mandatory whenever calling the constructor:
* freq specifies the maximum granularity at which injection or withdrawal decision are made. For more details see [this](#freq_argument) section below.
* storage_start is the first period where the commodity can be injected into, or withdrawn from storage. TODO link to section below on different ways to specify.
* storage_end is the period AFTER the last period where commodity and with injected or withdrawn. In the case where the storage has non-zero terminal value of inventory, it is on this period where the terminal value is evaluated. TODO link to section on terminal value.
* injection_cost is the cost per volume unit incurred upon increasing inventory by moving commodity into the storage. In the context of gas storage this is known as 'injection'.
* withdrawal_cost is the cost per volume unit incurred upon moving commodity out of storage, hence reducing the inventory. For gas storage this is known as 'withdrawal'.

The following argument are mandatory unless the storage is created with ratchets. TODO link to ratchets section below.
* min_inventory is the minimum volume of inventory which must be held in storage at any time. Most commonly this was have a value of zero.
* max_inventory is the maximum volume of inventory that can be held in storage at any time.
* max_injection_rate is the maximum volume of commodity which can be taken out of storage in any time period.
* max_withdrawal_rate is the maximum volume of commodity which can be taken out of storage in any time period.

### Time Varying Storage Properties
Although the last six argument to the CmdtyStorage constructor are used with scalar float values in the example above, Pandas Series can also be passed in to represent time varying storage characteristics for any of these. The code below gives an example of how this can be used to provide piecewise flat values which vary by year of a three year storage facility. This is relevant where storage is leased and as such a different nominal amount of storage facility is accessible per time period. Although not shown below min_inventory, max_injection_rate and max_withdrawal_rate can also be time-varying.

In [16]:
yearly_period_index = pd.PeriodIndex(freq='D', data = ['2021-04-01', '2022-04-01', '2023-04-01', '2024-04-01'])

time_varying_storage = CmdtyStorage(
    freq='D',
    storage_start = '2021-04-01',
    storage_end = '2024-04-01',
    injection_cost = pd.Series([0.01, 0.011, 0.013, 0.013], yearly_period_index).resample('D').fillna('pad'),
    withdrawal_cost = pd.Series([0.025, 0.026, 0.027, 0.027], yearly_period_index).resample('D').fillna('pad'),
    min_inventory = 0.0,
    max_inventory = pd.Series([1000, 1300, 800, 800], yearly_period_index).resample('D').fillna('pad'),
    max_injection_rate = 25.5,
    max_withdrawal_rate = 30.9
)

### Freq Argument: Nomination Granularity
<a id='freq_argument'></a>
The freq argument in the CmdtyStorage constructor represents the granularity at which storage injection and withdrawal exercise decisions are made and correspond to Pandas frequency strings as described [here](
https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects). For example if the highest granularity at which the final injection or withdrawal decision can be nominated is daily, use 'D'. The value used is also of significance where other arguments to the CmdtyStorage constructor (described below), arguments to the valuation functions, and attributes of the results from valuation function are of Pandas types Series or DataFrame. In these cases the arguments with have an index of PeriodIndex type with frequency that corresponds to this frequency string.

The package level dict variable FREQ_TO_PERIOD_TYPE contains a mapping between freq parameter values and the underlying managed types used to represent the resulting curve index type, and hence granularity. As such, the keys of FREQ_TO_PERIOD_TYPE can be used to determine the set of admissible values for the freq parameter.

In [6]:
from cmdty_storage import FREQ_TO_PERIOD_TYPE
FREQ_TO_PERIOD_TYPE

{'15min': Cmdty.TimePeriodValueTypes.QuarterHour,
 '30min': Cmdty.TimePeriodValueTypes.HalfHour,
 'H': Cmdty.TimePeriodValueTypes.Hour,
 'D': Cmdty.TimePeriodValueTypes.Day,
 'M': Cmdty.TimePeriodValueTypes.Month,
 'Q': Cmdty.TimePeriodValueTypes.Quarter}